In [ ]:
import torch
import numpy as np

In [ ]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [ ]:
x_ones = torch.ones_like(x_data)

In [ ]:
x_rand = torch.rand_like(x_data, dtype=torch.float)

In [ ]:
print(x_ones)
print(x_rand)

In [ ]:
print(x_rand.shape, x_rand.dtype, x_rand.device)

In [ ]:
if torch.cuda.is_available():
  x_rand = x_rand.to('cuda')

In [ ]:
print(x_rand.device)

In [ ]:
# mutrix multiplication
y1 = x_rand @ x_rand.T

In [ ]:
# element-wise multiplication
y2 = x_rand * x_rand

In [ ]:
print(y1)
print(y2)

In [ ]:
y1.sum().item()

# DataLoader

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

In [ ]:
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
  sample_idx = torch.randint(len(training_data), size=(1,)).item()
  img, label = training_data[sample_idx]
  figure.add_subplot(rows, cols, i)
  plt.title(labels_map[label])
  plt.axis('off')
  plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
train_images, train_labels = next(iter(train_dataloader))
print(train_images.size(), train_labels.size())

In [ ]:
img = train_images[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap='gray')
plt.show()
print(labels_map[label.item()])

# Transforms

In [ ]:
from torchvision.transforms import Lambda

In [ ]:
ds = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

# Build the neural network

In [ ]:
import os
from torch import nn
from torchvision import datasets, transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print('Predicted class:', y_pred)

In [ ]:
for name, param in model.named_parameters():
  print('Layer: {} | Size {} | Values {}'.format(name, param.size(), param[:2]))

# Automatic differentiation

In [ ]:
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print(w.grad_fn)
print(z.grad_fn)

In [ ]:
print(loss.grad_fn)

In [ ]:
loss.backward()

In [ ]:
print(w.grad)
print(b.grad)

In [ ]:
with torch.no_grad():
  z_no_grad = torch.matmul(x, w) + b
print(z_no_grad.requires_grad)

# Optimizing the model

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 3

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
  size = len(dataloader.dataset)
  for batch, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)
    pred = model(x)
    loss = loss_fn(pred, y)

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(x)
      print('loss: {:.2f} [{} / {}]'.format(loss, current, size))

In [ ]:
def test_loop(dataloader, model, loss_fn, device):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)
      pred = model(x)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print('Test error\n Accuracy: {:.1f}, Avg loss: {:.8f}'.format(100 * correct, test_loss))


In [ ]:
for t in range(epochs):
  print('Epoch {}\n----------'.format(t+1))
  train_loop(train_dataloader, model, loss_fn, optimizer, device)
  test_loop(test_dataloader, model, loss_fn, device)
print('Done')

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
restored_model = NeuralNetwork()
restored_model.load_state_dict(torch.load('model_weights.pth'))
restored_model.eval()

In [ ]:
torch.save(model, 'model.pth')

In [ ]:
restored_model2 = torch.load('model.pth')

In [ ]:
restored_model2